# Load Libraries/Data

In [1]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_info = pd.read_csv('Data/superhero_info.csv')
df_info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"


In [3]:
df_powers = pd.read_csv('Data/superhero_powers.csv')
df_powers.head(2)

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."


# Part 1: Clean Files and Combine Into One DataFrame

This dataframe should have the following columns:
- Hero (Just the name of the Hero)
- Publisher
- Gender
- Eye color
- Race
- Hair color
- Height (numeric)
- Skin color
- Alignment
- Weight (numeric)
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    - Agility
    - Flight
    - Superspeed
    - etc.
- Hint: There is a space in "100 kg" or "52.5 cm"

## Superhero_Info DataFrame

### Split 'Hero|Publisher' Column

In [4]:
df_info[['Hero', 'Publisher']] = df_info['Hero|Publisher'].str.split('|', expand=True)
df_info

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics
...,...,...,...,...,...,...,...,...,...,...
458,Yellowjacket|Marvel Comics,Male,Human,good,Blond,blue,Unknown,"{'Height': '183.0 cm', 'Weight': '83.0 kg'}",Yellowjacket,Marvel Comics
459,Yellowjacket II|Marvel Comics,Female,Human,good,Strawberry Blond,blue,Unknown,"{'Height': '165.0 cm', 'Weight': '52.0 kg'}",Yellowjacket II,Marvel Comics
460,Yoda|George Lucas,Male,Yoda's species,good,White,brown,green,"{'Height': '66.0 cm', 'Weight': '17.0 kg'}",Yoda,George Lucas
461,Zatanna|DC Comics,Female,Human,good,Black,blue,Unknown,"{'Height': '170.0 cm', 'Weight': '57.0 kg'}",Zatanna,DC Comics


In [5]:
# drop original column
df_info.drop(columns='Hero|Publisher', inplace=True)
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


### Set 'Hero' and 'Publisher' as First and Second Column

In [6]:
print(list(df_info.columns))

['Gender', 'Race', 'Alignment', 'Hair color', 'Eye color', 'Skin color', 'Measurements', 'Hero', 'Publisher']


In [7]:
df_info = df_info[['Hero', 'Publisher', 'Gender', 'Race', 'Alignment', 'Hair color', 'Eye color', 'Skin color', 'Measurements']]
df_info.head(2)

,Hero,Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb,Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien,Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"


### Split 'Measurements' Column

In [8]:
type(df_info['Measurements'][0])

str

In [9]:
df_info['Measurements'] = df_info['Measurements'].str.replace("'", '"')
df_info['Measurements']

0      {"Height": "203.0 cm", "Weight": "441.0 kg"}
1       {"Height": "191.0 cm", "Weight": "65.0 kg"}
2       {"Height": "185.0 cm", "Weight": "90.0 kg"}
3      {"Height": "203.0 cm", "Weight": "441.0 kg"}
4      {"Height": "193.0 cm", "Weight": "122.0 kg"}
                           ...                     
458     {"Height": "183.0 cm", "Weight": "83.0 kg"}
459     {"Height": "165.0 cm", "Weight": "52.0 kg"}
460      {"Height": "66.0 cm", "Weight": "17.0 kg"}
461     {"Height": "170.0 cm", "Weight": "57.0 kg"}
462     {"Height": "185.0 cm", "Weight": "81.0 kg"}
Name: Measurements, Length: 463, dtype: object

In [10]:
df_info['Measurements'] = df_info['Measurements'].apply(json.loads)
df_info['Measurements'].head(2)

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
Name: Measurements, dtype: object

In [11]:
type(df_info['Measurements'][0])

dict

In [12]:
height_weight = df_info['Measurements'].apply(pd.Series)
height_weight

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [13]:
df_info = pd.concat((df_info, height_weight), axis=1)
df_info.head(2)

,Hero,Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Height,Weight
0,A-Bomb,Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",203.0 cm,441.0 kg
1,Abe Sapien,Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",191.0 cm,65.0 kg


In [14]:
df_info.drop(columns='Measurements', inplace=True)
df_info.head(2)

,Hero,Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Height,Weight
0,A-Bomb,Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,203.0 cm,441.0 kg
1,Abe Sapien,Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,191.0 cm,65.0 kg


### Convert 'Height' and 'Weight' to Numeric Type

In [15]:
df_info.rename(columns={'Height': 'Height (cm)', 'Weight': 'Weight (kg)'},
              inplace=True)
df_info.head(2)

,Hero,Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Height (cm),Weight (kg)
0,A-Bomb,Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,203.0 cm,441.0 kg
1,Abe Sapien,Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,191.0 cm,65.0 kg


#### Height

In [16]:
type(df_info['Height (cm)'][0])

str

In [ ]:
# remove empty space and 'cm' from 'Height (cm)'

to_replace = [' ', 'cm']

for char in to_replace:
    df_info['Height (cm)'] = df_info['Height (cm)'].str.replace(char, '', regex=False)
df_info.head(2)

In [ ]:
type(df_info['Height (cm)'][0])

In [ ]:
df_info['Height (cm)'] = df_info['Height (cm)'].astype('float')
type(df_info['Height (cm)'][0])

#### Weight

In [ ]:
type(df_info['Weight (kg)'][0])

In [ ]:
# remove empty space and 'cm' from 'Height (cm)'

to_replace = [' ', 'kg']

for char in to_replace:
    df_info['Weight (kg)'] = df_info['Weight (kg)'].str.replace(char, '', regex=False)
df_info.head(2)

In [ ]:
df_info['Weight (kg)'] = df_info['Weight (kg)'].astype('float')
type(df_info['Weight (kg)'][0])

## Superhero_Powers DataFrame

In [ ]:
df_powers.head(2)

In [ ]:
print(type(df_powers['Powers'][0]))
df_powers['Powers'][0]

### Convert 'Powers' to List

In [ ]:
df_powers['Powers_Split'] = df_powers['Powers'].str.split(',')

In [ ]:
df_powers.head(2)

In [ ]:
print(type(df_powers['Powers_Split'][0]))
df_powers['Powers_Split']

### One-Hot-Encode 'Powers'

In [ ]:
df_powers['Powers_Split'].value_counts()

In [ ]:
## exploding the column of lists
exploded = df_powers.explode('Powers_Split')
exploded.head(5)

In [ ]:
## dropping null values first
## saving the unique values from the exploded column
cols_to_make = exploded['Powers_Split'].dropna().unique()
cols_to_make

In [ ]:
# making new column for each unique power from 'Power_Split' column
# checking with original 'Powers' column for True or False

for col in cols_to_make:
    df_powers[col] = df_powers['Powers'].str.contains(col)
df_powers.head()

In [ ]:
df_powers.drop(columns=['Powers', 'Powers_Split'], inplace=True)
df_powers.head(2)

## Combine Into One Dataframe

In [ ]:
df_final = pd.merge(df_info, df_powers, left_on='Hero', right_on='hero_names')
df_final.head(2)

#### Drop 'hero_names' Since It's a Duplicate of 'Hero'

In [ ]:
df_final.drop(columns='hero_names', inplace=True)
df_final.head(2)

# Part 2: Answer Questions

## 1. Compare the average weight of super powers (heroes?) who have Super Speed to those who do not.

In [ ]:
np.where(cols_to_make == 'Super Speed')

In [ ]:
heroes_speed = df_final['Super Speed'] == True
df_speed = df_final[heroes_speed]
df_speed

In [ ]:
df_not_speed = df_final[~heroes_speed]
df_not_speed

In [ ]:
avg_speed_weight = df_speed['Weight (kg)'].mean().round(2)
avg_not_speed_weight = df_not_speed['Weight (kg)'].mean().round(2)
print(avg_speed_weight)
print(avg_not_speed_weight)

In [ ]:
print(f'The average weight (kg) for Super Heroes that have "Super Speed" is \
{avg_speed_weight} compared to the average weight of Super Heroes that don\'t \
{avg_not_speed_weight}')
print(f'On average, "Super Speed" Super Heroes weigh \
{(avg_speed_weight-avg_not_speed_weight).round(2)} kg \
more.')

In [ ]:
speed_dict = {'Have Super Speed': avg_speed_weight,
             'Do Not Have Super Speed': avg_not_speed_weight}

In [ ]:
plt.bar(speed_dict.keys(), speed_dict.values())
plt.xlabel('Fast or Slow')
plt.ylabel('Average Weight (kg)')
plt.title('Average Weight (kg) of Fast and Slow Super Heroes');

## 2. What is the average height of heroes for each publisher?

In [ ]:
publisher_height = df_final.groupby('Publisher')['Height (cm)'].mean().sort_values(ascending=False)
publisher_height

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax = plt.bar(publisher_height.index, publisher_height.values)
plt.xticks(rotation=45)
plt.grid(axis='y')

plt.xlabel('Publisher')
plt.ylabel('Average Height (cm)')
plt.title('Average Height (cm) per Publisher');